In [22]:
import pandas as pd
dfFivePoints = pd.read_csv('Data/fivePoint180Sec.csv')

In [23]:
dfFivePoints.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186405 entries, 0 to 186404
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   student_id     186405 non-null  int64  
 1   patternsm      186405 non-null  object 
 2   statuscl       176938 non-null  object 
 3   numbercl       176938 non-null  object 
 4   timestampcl    176937 non-null  object 
 5   timestampload  186405 non-null  float64
 6   timestampsm    186405 non-null  float64
 7   primary_keys   186405 non-null  object 
 8   datetime       186405 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 12.8+ MB


In [24]:
dfFivePoints.head()

,student_id,patternsm,statuscl,numbercl,timestampcl,timestampload,timestampsm,primary_keys,datetime
0,1,"0,0,1,0,0,0,1,0","1,1","3,7","1583742751461,1583742751869",1.583743e+12,2520.0,"[36, 37, 38, 39, 40, 41]",2020-03-09 08:32:29.790
1,1,"1,1,0,0,0,0,0,0","1,1","2,1","1583742753766,1583742754238",1.583743e+12,2576.0,"[47, 48, 49, 50, 51, 52]",2020-03-09 08:32:32.391
2,1,"0,0,1,1,0,0,0,0","1,1","3,4","1583742756412,1583742756876",1.583743e+12,2541.0,"[53, 54, 55, 56, 57, 58]",2020-03-09 08:32:35.058
3,1,"0,0,0,0,0,1,0,1","1,1","8,6","1583742759029,1583742759476",1.583743e+12,2540.0,"[59, 60, 61, 62, 63, 64]",2020-03-09 08:32:37.674
4,1,"0,0,1,1,0,0,0,0","1,1","4,3","1583742761381,1583742762222",1.583743e+12,2384.0,"[65, 66, 67, 68, 69, 70]",2020-03-09 08:32:40.309


Here we use the cleaned up of the csv and as you can see it show each pattern base on the student_id and other additional information.

In [25]:
rows = dfFivePoints['student_id'].value_counts().reset_index()
rows.columns = ['student_id', 'total_patterns']

# Groepeer de rijen op basis van 'student_id' en neem de som van de 'pattern' waar deze gelijk is aan "0,0,0,0,0,0,0,0"
sum_of_pattern = dfFivePoints[dfFivePoints['patternsm'] == "0,0,0,0,0,0,0,0"].groupby('student_id')['patternsm'].count().reset_index()
sum_of_pattern.columns = ['student_id', 'sum_invalid_pattern']

total_duplicates = dfFivePoints.groupby('student_id').size().reset_index()
total_duplicates.columns = ['student_id', 'total_duplicates']

# Voeg de sommen toe aan de oorspronkelijke DataFrame
rows = pd.merge(rows, sum_of_pattern, on='student_id', how='left')

rows

,student_id,total_patterns,sum_invalid_pattern
0,879,209,192.0
1,1161,175,156.0
2,748,164,137.0
3,9884,150,66.0
4,2639,137,77.0
...,...,...,...
4545,51,1,NaN
4546,4376,1,NaN
4547,1354,1,1.0
4548,9887,1,1.0


We decided to make a new csv that has a summed up result of what each student did base on the csv given. As you can see here we get the created 2 new columns where it show the "total_patterns" and "sum_invalid_pattern".
<br>
<br>
The column "total_patterns" means the total pattern/figures made by the student including duplicates and incorrect one's.<br>
The column "sum_invalid_pattern" means the total amount of patterns that were invalid meaning patternsm = '0,0,0,0,0,0,0,0'. So no line was selected

In [26]:
# Filter out rows with 'patternsm' not equal to '0,0,0,0,0,0,0,0'
non_zero_df = dfFivePoints[dfFivePoints['patternsm'] != '0,0,0,0,0,0,0,0']

# Group by 'student_id' and count the unique patterns
unique_patterns_per_student = non_zero_df.groupby('student_id')['patternsm'].nunique().reset_index()
unique_patterns_per_student.columns = ['student_id', 'unique_patterns_count']

# Calculate the total count of values for each student, including zeros
total_values_per_student = dfFivePoints.groupby('student_id')['patternsm'].count().reset_index()
total_values_per_student.columns = ['student_id', 'total_values_count']

# Calculate the total count of values for each student, excluding zeros
total_non_zero_values_per_student = non_zero_df.groupby('student_id')['patternsm'].count().reset_index()
total_non_zero_values_per_student.columns = ['student_id', 'total_non_zero_values_count']

# Merge the DataFrames to create the new DataFrame
new_df = pd.merge(unique_patterns_per_student, total_values_per_student, on='student_id')
new_df = pd.merge(new_df, total_non_zero_values_per_student, on='student_id')

# Calculate the "duplicates" column
new_df['duplicates'] = new_df['total_non_zero_values_count'] - new_df['unique_patterns_count']

new_df.head()

,student_id,unique_patterns_count,total_values_count,total_non_zero_values_count,duplicates
0,1,4,5,5,1
1,2,26,28,28,2
2,3,38,48,47,9
3,4,54,64,62,8
4,5,48,63,62,14


Here we made a bunch more colums/features such as unique_patterns_count which is how many good patterns they made.
<br>
"total_values_count" shows how many pattern they made including duplicated and incorrect/zero patterns
<br>
"total_non_zero_values_count" is the colums that show all the patterns including the duplicated but excapt the zero/incorrect patterns

In [27]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4534 entries, 0 to 4533
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   student_id                   4534 non-null   int64
 1   unique_patterns_count        4534 non-null   int64
 2   total_values_count           4534 non-null   int64
 3   total_non_zero_values_count  4534 non-null   int64
 4   duplicates                   4534 non-null   int64
dtypes: int64(5)
memory usage: 177.2 KB


In [28]:
# combine the DataFrames
combine = pd.merge(rows, new_df, on='student_id', how='left')

combine.head()

,student_id,total_patterns,sum_invalid_pattern,unique_patterns_count,total_values_count,total_non_zero_values_count,duplicates
0,879,209,192.0,6.0,209.0,17.0,11.0
1,1161,175,156.0,17.0,175.0,19.0,2.0
2,748,164,137.0,25.0,164.0,27.0,2.0
3,9884,150,66.0,28.0,150.0,84.0,56.0
4,2639,137,77.0,16.0,137.0,60.0,44.0


Here we combine both dataframe but the the total_patterns will be dropped for as it's the same as the total_values_count

In [29]:
combine = combine.drop('total_patterns', axis=1)

In [30]:
combine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4550 entries, 0 to 4549
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   student_id                   4550 non-null   int64  
 1   sum_invalid_pattern          2703 non-null   float64
 2   unique_patterns_count        4534 non-null   float64
 3   total_values_count           4534 non-null   float64
 4   total_non_zero_values_count  4534 non-null   float64
 5   duplicates                   4534 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 213.4 KB


In [31]:
#check student_id 1 in df combine
combine[combine['student_id'] == 1]

,student_id,sum_invalid_pattern,unique_patterns_count,total_values_count,total_non_zero_values_count,duplicates
4514,1,NaN,4.0,5.0,5.0,1.0


Here is the example how it would look like for student one but therer is some na fills that needs to be cleaned up.

In [32]:
combine.describe()

,student_id,sum_invalid_pattern,unique_patterns_count,total_values_count,total_non_zero_values_count,duplicates
count,4550.000000,2703.000000,4534.000000,4534.000000,4534.000000,4534.000000
mean,5218.844615,3.677026,34.409131,41.101015,38.920600,4.511469
std,3034.395206,7.623618,12.733502,15.209543,14.237742,4.957683
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,2619.250000,1.000000,26.000000,31.000000,29.000000,2.000000
50%,5223.500000,2.000000,35.000000,41.000000,39.000000,3.000000
75%,8020.750000,4.000000,43.000000,50.000000,48.000000,6.000000
max,10058.000000,192.000000,82.000000,209.000000,113.000000,81.000000


In [33]:
# check for na values
combine.isna().sum()


student_id                        0
sum_invalid_pattern            1847
unique_patterns_count            16
total_values_count               16
total_non_zero_values_count      16
duplicates                       16
dtype: int64

Here we check all the na values

In [34]:
#show the rows with total_values_count is nan
combine[combine['total_values_count'].isna()]

,student_id,sum_invalid_pattern,unique_patterns_count,total_values_count,total_non_zero_values_count,duplicates
4485,2060,7.0,NaN,NaN,NaN,NaN
4496,65,6.0,NaN,NaN,NaN,NaN
4501,8159,6.0,NaN,NaN,NaN,NaN
4510,9782,5.0,NaN,NaN,NaN,NaN
4520,9781,4.0,NaN,NaN,NaN,NaN
4521,9743,4.0,NaN,NaN,NaN,NaN
4523,9768,4.0,NaN,NaN,NaN,NaN
4527,5156,3.0,NaN,NaN,NaN,NaN
4528,2690,3.0,NaN,NaN,NaN,NaN
4529,9849,3.0,NaN,NaN,NaN,NaN


We decided to drop these rows as these are invalid pattern therefore it's not needed

In [35]:
#drop the rows with total_values_count is nan
combine = combine.dropna(subset=['total_values_count'])

combine.describe()


,student_id,sum_invalid_pattern,unique_patterns_count,total_values_count,total_non_zero_values_count,duplicates
count,4534.000000,2687.000000,4534.000000,4534.000000,4534.000000,4534.000000
mean,5212.932730,3.679196,34.409131,41.101015,38.920600,4.511469
std,3031.091136,7.644792,12.733502,15.209543,14.237742,4.957683
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,2618.250000,1.000000,26.000000,31.000000,29.000000,2.000000
50%,5217.000000,2.000000,35.000000,41.000000,39.000000,3.000000
75%,8003.750000,4.000000,43.000000,50.000000,48.000000,6.000000
max,10058.000000,192.000000,82.000000,209.000000,113.000000,81.000000


In [36]:
#fill sum_invalid_pattern with 0
combine['sum_invalid_pattern'] = combine['sum_invalid_pattern'].fillna(0)

combine.describe()


,student_id,sum_invalid_pattern,unique_patterns_count,total_values_count,total_non_zero_values_count,duplicates
count,4534.000000,4534.000000,4534.000000,4534.000000,4534.000000,4534.000000
mean,5212.932730,2.180415,34.409131,41.101015,38.920600,4.511469
std,3031.091136,6.156185,12.733502,15.209543,14.237742,4.957683
min,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,2618.250000,0.000000,26.000000,31.000000,29.000000,2.000000
50%,5217.000000,1.000000,35.000000,41.000000,39.000000,3.000000
75%,8003.750000,2.000000,43.000000,50.000000,48.000000,6.000000
max,10058.000000,192.000000,82.000000,209.000000,113.000000,81.000000


Here we decided to fill in the NaN values for "sum_invalid_pattern", because if they have NaN incorrect value means they made 0 incorrect values

In [37]:
#check student_id 1 in df combine
combine[combine['student_id'] == 1]

,student_id,sum_invalid_pattern,unique_patterns_count,total_values_count,total_non_zero_values_count,duplicates
4514,1,0.0,4.0,5.0,5.0,1.0


In [38]:
# check for na values
combine.isna().sum()

student_id                     0
sum_invalid_pattern            0
unique_patterns_count          0
total_values_count             0
total_non_zero_values_count    0
duplicates                     0
dtype: int64

Here is there is no more NaN values left and will be ready to use for creating more features and labeling them